# Hill Climbing Method

## Pseudo code

```
Initialize the weights 𝜃 in the policy arbitarily.

Collect an episode with 𝜃, and record the return G.

𝜃_best <-- 𝜃, G_best <-- G

Repeat until environment solved:

    Add a little bit of random noise to 𝜃_best, to get a new set of weight 𝜃_new.

    Collect an episode with 𝜃_new, and record the return G_new.
    
    if G_new < G_best, then:
        𝜃_best <-- 𝜃_new, G_best <-- G_new
```


### Import the Necessary Packages